In [1]:
dict = {'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805}

key_words = ['account', 'member', 'access', 'email', 'address',
             'update', 'price', 'market', 'online', 'information',
             'work', 'credit', 'response', 'offer', 'transaction',
             'agreement', 'registration', 'person', 'system',
             'process', 'service', 'request', 'message']

key_weights = {'HTML Body':0.681, 'Hexadecimal URLs':0.652, 'Domains Count':0.652, 'TextLinkDifference':0.649, 'Dots Count':0.479,
                'Email Contains Account Term':0.493, 'Email Contains Dear Term':0.375, 'Images as URL':0.298, 'IP URLs':0.297,
                'Email Contains PayPal Term':0.296, 'Email Contains Login Term':0.250, 'Email Contains Bank Term':0.213, 
                'Phishing Terms Weight':0.210, 'Email Contains Verify Term':0.207,
                'Email Contains Agree Term':0.206, 'Email Contains Suspend Term':0.205}

key_features = ['HTML Body', 'Hexadecimal URLs', 'Domains Count', 'TextLinkDifference', 'Dots Count',
                'Email Contains Account Term', 'Email Contains Dear Term', 'Images as URL', 'IP URLs',
                'Email Contains PayPal Term', 'Email Contains Login Term', 'Email Contains Bank Term',
                'Phishing Terms Weight', 'Email Contains Verify Term',
                'Email Contains Agree Term', 'Email Contains Suspend Term']

print("key_weights length:" + str(len(key_weights)))
print("key_features length:" + str(len(key_features)))

key_weights length:16
key_features length:16


In [2]:
#read data from csv
import pandas as pd
train_data = pd.read_csv('../output/train_1.csv')
test_data = pd.read_csv('../output/train_2.csv')


In [3]:
train_pd = pd.DataFrame(train_data)
train_pd.head()


,Unnamed: 0,0,1
0,0,Looks good to me. Are we certain that all of ...,0
1,1,"Greg, how big was that last check you told me ...",0
2,2,"<table border=0 width=""100%"" cellpadding=""8"" ...",1
3,3,Hello everyone:\n\nIf you were not able to acc...,0
4,4,"I have a customer in town Thursday night, so w...",0


In [4]:
test_pd = pd.DataFrame(test_data)
test_pd.head()


,Unnamed: 0,0,1
0,0,"Janet,\n\nHere is the model. I have added uni...",0
1,1,"Maureen,\n\nYour presentation:\n\nVince",0
2,2,"Kim, please invite the Following Texas Eastern...",0
3,3,"\n<br>\n\n<p dir=""ltr"" align=""left""><b><font c...",1
4,4,noregrets littlerock2016 alex hoffman democrat...,0


In [2]:
import urllib.request
from pyquery import PyQuery
from bs4 import BeautifulSoup
import regex
from tld import get_tld
import tldextract
import ipaddress
import whois

def is_ip_address(url):
    try:
        ipaddress.ip_address(url)
        return 1
    except:
        return -1

def Domain_registration_length(url):
    try:
        whois_database = whois.query(url)
        updated_datetime = whois_database.last_updated
        if updated_datetime is None:
            updated_datetime = whois_database.creation_date
        # print(updated_datetime)
        exp_datetime = whois_database.expiration_date
        # print(exp_datetime)
        length_period = (exp_datetime - updated_datetime).days
        # print(length_period)
        if (length_period <= 365):
            return -1
        else:
            return 1
    except:
        return 0


In [6]:
# print(Domain_registration_length('https://www.baidu.com'))

In [3]:
class EmailFeaturized(object):
    def __init__(self, content):
        self.content = str(content).lower()
        self.urls = []
        self.domains = []
    
    def is_html_body(self):
        if BeautifulSoup(self.content, "html.parser").find():
            return 1
        else:
            return 0
    
    def num_Hexadecimal_urls(self):
        self.get_urls()
        self.get_links()
        hex_url_count = 0
        for domain in set(self.domains):
            if self.Hexadecimal_urls(domain):
                hex_url_count = hex_url_count + 1
        return hex_url_count
    
    # 16进制URL
    def Hexadecimal_urls(self, url):
        match = regex.search(
            '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
            '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  # IPv4 in hexadecimal
            '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
        if match:
            # print match.group()
            return -1
        else:
            # print 'No matching pattern found'
            return 1
        
    def get_urls(self):
        self.urls = []
        try:
            soup = BeautifulSoup(self.content)
        except Exception as e:
            # print('unknown error %s' %e)
            return []
        for a in soup.find_all('a'):
            try:
                url = a['href']
                self.urls.append(url)
            except Exception as e:
                return self.urls
        return self.urls
        
    def get_links(self):
        self.domains = []
        try:
            soup = BeautifulSoup(self.content)
        except Exception as e:
            # print('unknown error %s' %e)
            return []
        for a in soup.find_all('a'):
            try:
                url = a['href']
                # domain = get_tld(url)
                # print(get_tld('https://www.baidu.com'))
                val = tldextract.extract(url)
                domain = "{0}.{1}".format(val.domain, val.suffix)
                self.domains.append(domain)
            except Exception as e:
                return self.domains
        return self.domains
    
    def num_domain(self):
        self.get_urls()
        self.get_links()
        return len(set(self.domains))
    
    def text_link_difference(self):
        # ongoing
        return 0
    
    def dots_counts(self):
        num = 0
        for url in self.urls:
            if url.count('.') > num:
                num = url.count('.')
        return num
    
    def contain_account(self):
        if 'account' in self.content:
            return 1
        return 0
    
    def contain_dear(self):
        if 'dear' in self.content:
            return 1
        return 0
    
    def numbers_image_urls(self):
        soup = BeautifulSoup(self.content)
        images = soup.findAll('img', src=True)
        total = len(images)
        return total
    
    def numbers_of_ip_urls(self):
        self.get_urls()
        self.get_links()
        num = 0
        for domain in self.domains:
            if is_ip_address(domain) == 1:
                num = num + 1
        return num
    
    def contain_paypal(self):
        if 'paypal' in self.content:
            return 1
        return 0
    
    def contain_login(self):
        if 'login' in self.content:
            return 1
        return 0
    
    def contain_bank(self):
        if 'bank' in self.content:
            return 1
        return 0
    
    # should change it to tf-idf
    def phishing_weight(self):
        num = 0.0
        for word in key_words:
            if word in self.content:
                num += float(dict[word])
        return num
    
    def contain_verify(self):
        if 'verify' in self.content:
            return 1
        return 0
    
    def contain_agree(self):
        if 'agree' in self.content:
            return 1
        return 0
    
    def contain_suspend(self):
        if 'suspend' in self.content:
            return 1
        return 0
    
    def remove_stop_words(self):
        pass
    
    def stemming(self):
        pass
    
    def domain_age_less_than_365(self):
        for domain in self.domains:
            age_level = Domain_registration_length(domain)
            if age_level != 1:
                return age_level
        return 1
    
    def run(self):
        data = {}
        data['HTML Body'] = self.is_html_body()
        data['Hexadecimal URLs'] = self.num_Hexadecimal_urls()
        data['Domains Count'] = self.num_domain()
        data['TextLinkDifference'] = self.text_link_difference()
        data['Dots Count'] = self.dots_counts()
        data['Email Contains Account Term'] = self.contain_account()
        data['Email Contains Dear Term'] = self.contain_dear()
        data['Images as URL'] = self.numbers_image_urls()
        data['IP URLs'] = self.numbers_of_ip_urls()
        data['Email Contains PayPal Term'] = self.contain_paypal()
        data['Email Contains Login Term'] = self.contain_login()
        data['Email Contains Bank Term'] = self.contain_bank()
        data['Phishing Terms Weight'] = self.phishing_weight()
        data['Email Contains Verify Term'] = self.contain_verify()
        data['Email Contains Agree Term'] = self.contain_agree()
        data['Email Contains Suspend Term'] = self.contain_suspend()
        return data
        

In [8]:
train_pd["HTML Body"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).is_html_body())

/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "http://www.potterybarn.com/cat/pip.cfm?grp=151094&src=schfname%20ion%20nlexi
ngton%20rdes" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "http://games.yahoo.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "https://www.columbianavigator.com/ebb/default.asp" looks like a URL. Beautiful Soup is not 

In [9]:
train_pd["Hexadecimal URLs"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).num_Hexadecimal_urls())

In [10]:
train_pd["Domains Count"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).num_domain())

In [11]:
train_pd["TextLinkDifference"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).text_link_difference())

In [12]:
train_pd["Dots Count"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).dots_counts())

In [13]:
train_pd["Email Contains Account Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_account())

In [14]:
train_pd["Email Contains Dear Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_dear())

In [15]:
train_pd["Images as URL"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).numbers_image_urls())

In [16]:
train_pd["IP URLs"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).numbers_of_ip_urls())

In [17]:
train_pd["Email Contains PayPal Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_paypal())

In [18]:
train_pd["Email Contains Login Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_login())

In [19]:
train_pd["Email Contains Bank Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_bank())

In [20]:
train_pd["Phishing Terms Weight"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).phishing_weight())

In [21]:
train_pd["Email Contains Verify Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_verify())

In [22]:
train_pd["Email Contains Agree Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_agree())

In [23]:
train_pd["Email Contains Suspend Term"] = train_pd["0"].apply(lambda i : EmailFeaturized(i).contain_suspend())

In [24]:
train_pd.head()

,Unnamed: 0,0,1,HTML Body,Hexadecimal URLs,Domains Count,TextLinkDifference,Dots Count,Email Contains Account Term,Email Contains Dear Term,Images as URL,IP URLs,Email Contains PayPal Term,Email Contains Login Term,Email Contains Bank Term,Phishing Terms Weight,Email Contains Verify Term,Email Contains Agree Term,Email Contains Suspend Term
0,0,Looks good to me. Are we certain that all of ...,0,0,0,0,0,0,0,0,0,0,0,0,0,15.364725,0,1,0
1,1,"Greg, how big was that last check you told me ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0
2,2,"<table border=0 width=""100%"" cellpadding=""8"" ...",1,1,2,2,0,0,0,0,0,0,0,0,1,49.785970,0,0,0
3,3,Hello everyone:\n\nIf you were not able to acc...,0,0,0,0,0,0,0,0,0,0,0,0,0,17.667806,0,0,0
4,4,"I have a customer in town Thursday night, so w...",0,0,0,0,0,0,0,0,0,0,0,0,0,8.659211,0,0,0


In [25]:
# html_path = '../data/jose_phishing_html/phishing-2015.txt-0009.eml.html'
# text_path = '../data/jose_phishing_text/phishing-2015.txt-0009.eml.txt'
# raw_path = '../data/jose_phishing/'
# 
# text = ''
# with open(html_path, 'r') as f:
#     text = f.read()
#     
# email_featurize = EmailFeaturized(text)

# print(email_featurize.is_html_body())
# print(email_featurize.get_links())
# print(email_featurize.get_urls())
# print(email_featurize.num_domain())
# print(email_featurize.dots_counts())
# print(email_featurize.numbers_image_urls())
# print(email_featurize.numbers_of_ip_urls())
# print(email_featurize.phishing_weight())
# print(email_featurize.num_Hexadecimal_urls())

In [26]:
test_pd["HTML Body"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).is_html_body())
test_pd["Hexadecimal URLs"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).num_Hexadecimal_urls())
test_pd["Domains Count"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).num_domain())
test_pd["TextLinkDifference"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).text_link_difference())
test_pd["Dots Count"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).dots_counts())
test_pd["Email Contains Account Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_account())
test_pd["Email Contains Dear Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_dear())
test_pd["Images as URL"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).numbers_image_urls())
test_pd["IP URLs"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).numbers_of_ip_urls())
test_pd["Email Contains PayPal Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_paypal())
test_pd["Email Contains Login Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_login())
test_pd["Email Contains Bank Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_bank())
test_pd["Phishing Terms Weight"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).phishing_weight())
test_pd["Email Contains Verify Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_verify())
test_pd["Email Contains Agree Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_agree())
test_pd["Email Contains Suspend Term"] = test_pd["0"].apply(lambda i : EmailFeaturized(i).contain_suspend())


/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "http://www.housingfinance.org/country%20specific%20information/country_specific_information.htm" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "http://rcommerce.us.dell.com/rcomm/config.asp?order_code=rcrc968773-52962" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "http://www.rsn.com/pages/include/ca

In [27]:
test_pd.head()

,Unnamed: 0,0,1,HTML Body,Hexadecimal URLs,Domains Count,TextLinkDifference,Dots Count,Email Contains Account Term,Email Contains Dear Term,Images as URL,IP URLs,Email Contains PayPal Term,Email Contains Login Term,Email Contains Bank Term,Phishing Terms Weight,Email Contains Verify Term,Email Contains Agree Term,Email Contains Suspend Term
0,0,"Janet,\n\nHere is the model. I have added uni...",0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0
1,1,"Maureen,\n\nYour presentation:\n\nVince",0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0
2,2,"Kim, please invite the Following Texas Eastern...",0,0,0,0,0,0,0,0,0,0,0,0,0,15.327600,0,0,0
3,3,"\n<br>\n\n<p dir=""ltr"" align=""left""><b><font c...",1,1,2,2,0,0,1,1,0,0,0,1,0,87.125801,0,0,0
4,4,noregrets littlerock2016 alex hoffman democrat...,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0,0


In [28]:
import numpy as np


train_ = train_pd[key_features]
label_ = train_pd["1"]
    # .apply(lambda i : int(i))

test_ = test_pd[key_features]
label_t = test_pd["1"]
    # .apply(lambda i : int(i))

In [29]:
# train_.head()
# label_.head()
# test_.head()
# label_t.head()
# train_.dtypes["account"]
# label_t.dtypes["1"]
label_t.head()

0    0
1    0
2    0
3    1
4    0
Name: 1, dtype: int64

In [30]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression


In [32]:
import coremltools
from coremltools.converters import sklearn

#Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(train_, label_)

coreml_model = sklearn.convert(dt_model, key_features, 'output')
coreml_model.save('coreml_dt.mlmodel')

dt_predictions = dt_model.predict(test_)
accuracy_score(label_t, dt_predictions)

0.9451651731978066

In [33]:
model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'HTML Body':1, 'Hexadecimal URLs':1, 'Domains Count':1, 'TextLinkDifference':1, 'Dots Count':1,
                'Email Contains Account Term':1, 'Email Contains Dear Term':1, 'Images as URL':1, 'IP URLs':1,
                'Email Contains PayPal Term':1, 'Email Contains Login Term':1, 'Email Contains Bank Term':1, 
                'Phishing Terms Weight':1, 'Email Contains Verify Term':1,
                'Email Contains Agree Term':1, 'Email Contains Suspend Term':1})
print(pred_out_put["output"])
pred_out_put


1


0.9451651731978066

In [34]:
print(confusion_matrix(label_t, dt_predictions))
print(classification_report(label_t, dt_predictions))


[[10243   316]
 [  504  3891]]
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     10559
           1       0.92      0.89      0.90      4395

    accuracy                           0.95     14954
   macro avg       0.94      0.93      0.93     14954
weighted avg       0.94      0.95      0.94     14954



In [35]:

#Random Forest
rfc = RandomForestClassifier()
rfc.fit(train_, label_)

coreml_model_rf = sklearn.convert(rfc, key_features, 'output')
coreml_model_rf.save('coreml_rf.mlmodel')

dt_predictions = rfc.predict(test_)
accuracy_score(label_t, dt_predictions)

0.945900762337836

In [36]:
model = coremltools.models.MLModel('coreml_rf.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'HTML Body':1, 'Hexadecimal URLs':1, 'Domains Count':1, 'TextLinkDifference':1, 'Dots Count':1,
                'Email Contains Account Term':1, 'Email Contains Dear Term':1, 'Images as URL':1, 'IP URLs':1,
                'Email Contains PayPal Term':1, 'Email Contains Login Term':1, 'Email Contains Bank Term':1, 
                'Phishing Terms Weight':1, 'Email Contains Verify Term':1,
                'Email Contains Agree Term':1, 'Email Contains Suspend Term':1})
print(pred_out_put["output"])
pred_out_put

1


{'output': 1,
 'classProbability': {0: 0.009999999999999343, 1: 0.9900000000000007}}

In [37]:
print("Random Forest")
print(confusion_matrix(label_t, dt_predictions))
print(classification_report(label_t, dt_predictions))

Random Forest
[[10226   333]
 [  476  3919]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     10559
           1       0.92      0.89      0.91      4395

    accuracy                           0.95     14954
   macro avg       0.94      0.93      0.93     14954
weighted avg       0.95      0.95      0.95     14954



In [38]:
#Logistic Regression
log_model = LogisticRegression()
log_model.fit(train_, label_)

log_predictions = log_model.predict(test_)
accuracy_score(label_t, log_predictions)

/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9113949444964558

In [39]:
print("Logistic Regression")

print(confusion_matrix(label_t, log_predictions))

print(classification_report(label_t, log_predictions))


Logistic Regression
[[10191   368]
 [  957  3438]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     10559
           1       0.90      0.78      0.84      4395

    accuracy                           0.91     14954
   macro avg       0.91      0.87      0.89     14954
weighted avg       0.91      0.91      0.91     14954



In [40]:
from sklearn import tree

###create decision tree classifier object
DT = tree.DecisionTreeClassifier(criterion="gini", max_depth=4)
##fit decision tree model with training data
DT.fit(train_, label_)
##test data prediction
DT_expost_preds = DT.predict(test_)
accuracy_score(label_t, DT_expost_preds)



0.9176140163167046

In [41]:

print("gini decision tree classifier ")

print(confusion_matrix(label_t, DT_expost_preds))
print(classification_report(label_t, DT_expost_preds))

gini decision tree classifier 
[[10333   226]
 [ 1006  3389]]
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10559
           1       0.94      0.77      0.85      4395

    accuracy                           0.92     14954
   macro avg       0.92      0.87      0.89     14954
weighted avg       0.92      0.92      0.92     14954



In [42]:
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
dtree.fit(train_, label_)

coreml_model = sklearn.convert(dtree, key_features, 'output')
coreml_model.save('coreml_dtree.mlmodel')

pred = dtree.predict(test_)
accuracy_score(label_t, pred)


0.9157416075966297

In [43]:
model = coremltools.models.MLModel('coreml_dtree.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'HTML Body':1, 'Hexadecimal URLs':1, 'Domains Count':1, 'TextLinkDifference':1, 'Dots Count':1,
                'Email Contains Account Term':1, 'Email Contains Dear Term':1, 'Images as URL':1, 'IP URLs':1,
                'Email Contains PayPal Term':1, 'Email Contains Login Term':1, 'Email Contains Bank Term':1, 
                'Phishing Terms Weight':1, 'Email Contains Verify Term':1,
                'Email Contains Agree Term':1, 'Email Contains Suspend Term':1})
print(pred_out_put["output"])
pred_out_put

1


{'output': 1, 'classProbability': {0: 0.0, 1: 1.0}}

In [44]:
print("entropy decision tree classifier ")

print(confusion_matrix(label_t, pred))
print(classification_report(label_t, pred))

entropy decision tree classifier 
[[10351   208]
 [ 1052  3343]]
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10559
           1       0.94      0.76      0.84      4395

    accuracy                           0.92     14954
   macro avg       0.92      0.87      0.89     14954
weighted avg       0.92      0.92      0.91     14954



In [45]:
# from sklearn.svm import SVC
# only linearSVC can be converted 
from sklearn.svm import LinearSVC as SVC
# svc=SVC(kernel='sigmoid')
svc=SVC()
inputs = np.asarray(train_, dtype=float)
outputs = np.asarray(label_, dtype=int)
model_5=svc.fit(inputs, outputs)

coreml_model = sklearn.convert(model_5, key_features, 'output')
coreml_model.save('coreml_svm.mlmodel')

pred = model_5.predict(test_)
accuracy_score(label_t,pred)


/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9073826400962953

In [46]:
# from sklearn.svm import SVC
# only linearSVC can be converted 
from sklearn.svm import LinearSVC as SVC
# svc=SVC(kernel='sigmoid')
svc=SVC()
inputs = np.asarray(train_, dtype=float)
outputs = np.asarray(label_, dtype=int)
model_5=svc.fit(inputs, outputs)

coreml_model = sklearn.convert(model_5, key_features, 'output')
coreml_model.save('coreml_svm.mlmodel')

pred = model_5.predict(test_)
accuracy_score(label_t,pred)


/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9077838705363114

In [47]:
model = coremltools.models.MLModel('coreml_svm.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'HTML Body':1, 'Hexadecimal URLs':1, 'Domains Count':1, 'TextLinkDifference':1, 'Dots Count':1,
                'Email Contains Account Term':1, 'Email Contains Dear Term':1, 'Images as URL':1, 'IP URLs':1,
                'Email Contains PayPal Term':1, 'Email Contains Login Term':1, 'Email Contains Bank Term':1, 
                'Phishing Terms Weight':1, 'Email Contains Verify Term':1,
                'Email Contains Agree Term':1, 'Email Contains Suspend Term':1})
print(pred_out_put["output"])
pred_out_put

1


{'output': 1,
 'classProbability': {0: 0.02214861454736361, 1: 0.9778513854526364}}

In [48]:
print("svm ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t,pred))


svm 
[[10205   354]
 [ 1025  3370]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     10559
           1       0.90      0.77      0.83      4395

    accuracy                           0.91     14954
   macro avg       0.91      0.87      0.88     14954
weighted avg       0.91      0.91      0.91     14954



In [49]:
from sklearn.ensemble import AdaBoostClassifier
adc=AdaBoostClassifier(n_estimators=5,learning_rate=1)
model_6=adc.fit(train_, label_)

pred = model_6.predict(test_)
accuracy_score(label_t,pred)


0.905175872676207

In [50]:
print("AdaBoostClassifier ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t,pred))

AdaBoostClassifier 
[[10360   199]
 [ 1219  3176]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.94     10559
           1       0.94      0.72      0.82      4395

    accuracy                           0.91     14954
   macro avg       0.92      0.85      0.88     14954
weighted avg       0.91      0.91      0.90     14954



In [51]:
import numpy as np
from xgboost import XGBClassifier

xgb=XGBClassifier()
# model_7=xgb.fit(temp_train_data, temp_train_label)
# pred = model_7.predict(temp_test_data)
model_7=xgb.fit(np.asarray(train_, dtype=float), np.asarray(label_, dtype=int))
pred = model_7.predict(np.asarray(test_))


accuracy_score(label_t,pred)


0.9414872274976595

In [52]:
print("xgboost ")

print(confusion_matrix(label_t, pred))
print(classification_report(label_t, pred))

xgboost 
[[10357   202]
 [  673  3722]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     10559
           1       0.95      0.85      0.89      4395

    accuracy                           0.94     14954
   macro avg       0.94      0.91      0.93     14954
weighted avg       0.94      0.94      0.94     14954



In [53]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
model_gb=gb.fit(train_, label_)

# coreml_model = sklearn.convert(model_gb, key_words, 'output')
# coreml_model.save('coreml_gb.mlmodel')

pred = model_gb.predict(test_)
accuracy_score(label_t, pred)


0.9415540992376622

In [54]:
print("AdaBoostClassifier ")

print(confusion_matrix(label_t,pred))
print(classification_report(label_t, pred))


AdaBoostClassifier 
[[10362   197]
 [  677  3718]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     10559
           1       0.95      0.85      0.89      4395

    accuracy                           0.94     14954
   macro avg       0.94      0.91      0.93     14954
weighted avg       0.94      0.94      0.94     14954



In [4]:
#test method for normal

import os
files = []

for dirname, _, filenames in os.walk('../test_data/from_weicheng/normal_txt'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
files[0]    

"../test_data/from_weicheng/normal_txt/from your dvr's snap jpg.txt"

In [5]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data
    
file_contents = get_data_content(files)
len(file_contents)

20

In [8]:
import coremltools
from coremltools.converters import sklearn

num = 0
for i in file_contents:
    feature_data = EmailFeaturized(i).run()
    model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
    pred_out_put = model.predict(feature_data)
    if str(pred_out_put["output"]) != '0':
        # print(i)
        print(files[num])
    print(pred_out_put["output"])
    num = num + 1


0
0
0
0
0
0
0
0
0
0
If you did not grant access, you should check this activity and secure your account.
Check activity
You received this email to let you know about important changes to your Google Account and services.
© 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA
../test_data/from_weicheng/normal_txt/Security alert for your linked Google Account.txt
1
0
0
0
		Your account shuhao_z@yahoo.com is listed as the recovery email for zhangshuhao@gmail.com. Don't recognize this account? Click here
Google
Email - Edison Mail was granted access to your linked Google Account
	zhangshuhao@gmail.com

If you did not grant access, you should check this activity and secure your account.
Check activity
You received this email to let you know about important changes to your Google Account and services.
© 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA
../test_data/from_weicheng/normal_txt/Security alert.txt
1
0
		Your account chatplatform@edison.te

In [11]:
#test method for phishing

import os
files = []

for dirname, _, filenames in os.walk('../test_data/from_weicheng/phishing_txt'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
files[0]    

'../test_data/from_weicheng/phishing_txt/Edison Mail in the of the Best Email Apps for Android.txt'

In [12]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data
    
file_contents = get_data_content(files)
len(file_contents)

16

In [14]:
num = 0
for i in file_contents:
    feature_data = EmailFeaturized(i).run()
    model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
    pred_out_put = model.predict(feature_data)
    print(pred_out_put["output"])
    if str(pred_out_put["output"]) != '1':
        # print(i)
        print(files[num])
    num = num + 1



1
0
../test_data/from_weicheng/phishing_txt/CSAC Alarm for 2019-10-01 20_00_00 at Home.txt
0
../test_data/from_weicheng/phishing_txt/CONFIRMATION.txt
0
../test_data/from_weicheng/phishing_txt/neteasy.txt
1
0
../test_data/from_weicheng/phishing_txt/E-Ticket Confirmation-BRUEFC 08DEC.txt
1
0
../test_data/from_weicheng/phishing_txt/hotmail.txt
0
../test_data/from_weicheng/phishing_txt/4.txt
0
../test_data/from_weicheng/phishing_txt/[Email-macOS] Feedback from brian.walker@walkercriminallawyers.com.au.txt
0
../test_data/from_weicheng/phishing_txt/I will forward the video to your relatives, friends, and co workers.txt
1
0
../test_data/from_weicheng/phishing_txt/File _Copy  of July Bonus 30JULY2020.xls_  has been shared with you..txt
0
../test_data/from_weicheng/phishing_txt/Update your Email mailsupport@edison.tech now to stay active.txt
0
../test_data/from_weicheng/phishing_txt/ticket-180841.txt
0
../test_data/from_weicheng/phishing_txt/ComputerBase Downloads_ Nvidia PhysX System Software 